# ЛР №3

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

insurance = pd.read_csv('insurance.csv')
insurance.head(5)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


<dl>
<dt> Описание колонок:
<dd>age - возраст основного бенефициара </dd>
<dd>sex - пол страховщика </dd>
<dd>bmi - индекс массы тела, дающий представление о теле, весе, который относительно высок или низок по отношению к росту,
объективный индекс массы тела (кг / м ^ 2)</dd>
<dd>children - количество детей, охваченных медицинской страховкой </dd>
<dd>smoker - курильщик или нет </dd>
<dd>region - район проживания </dd>
<dd>charges - индивидуальные медицинские расходы, оплачиваемые медицинской страховкой </dd>
</dl>


Создаём таргет: коэффициент стоимости страховки в зависимости от bmi

In [76]:
insurance['charges_coef'] = insurance['charges'] / insurance['bmi']

Предсказывая таргет для новых объектов в будущем, мы не будем заранее знать **charges**.

Удалим эту колонку из датасета.

In [77]:
insurance = insurance.drop(['charges'], axis=1)

Переведем колонки sex и smoker во множество {0, 1}

In [78]:
insurance['sex'] = insurance['sex'].apply(lambda x: 0 if x == 'female' else 1)
insurance['smoker'] = insurance['smoker'].apply(lambda x: 0 if x == 'no' else 1)

Переведем колонку region во множество {0, 3}

In [79]:
region_dict = {'southeast': 0, 'southwest': 1, 'northwest': 2, 'northeast': 3}
insurance['region'] = insurance['region'].apply(lambda x: region_dict[x])
print(insurance)

      age  sex     bmi  children  smoker  region  charges_coef
0      19    0  27.900         0       1       1    605.194409
1      18    1  33.770         1       0       0     51.097196
2      28    1  33.000         3       0       0    134.832182
3      33    1  22.705         0       0       2    968.265607
4      32    1  28.880         0       0       2    133.893878
...   ...  ...     ...       ...     ...     ...           ...
1333   50    1  30.970         3       0       2    342.284414
1334   18    0  31.920         0       0       3     69.109674
1335   18    0  36.850         0       0       0     44.228860
1336   21    0  25.800         0       0       1     77.827326
1337   61    0  29.070         0       1       2   1002.454775

[1338 rows x 7 columns]


Сохраним датасет с категориальными признаками для использования его в градиентном бустинге

In [80]:
insurance['id'] = [i for i in range(len(insurance))]
insurance = insurance.set_index('id')
insurance_for_lgb = insurance.copy()
insurance_for_lgb.head()

,age,sex,bmi,children,smoker,region,charges_coef
id,,,,,,,
0,19,0,27.900,0,1,1,605.194409
1,18,1,33.770,1,0,0,51.097196
2,28,1,33.000,3,0,0,134.832182
3,33,1,22.705,0,0,2,968.265607
4,32,1,28.880,0,0,2,133.893878


In [81]:
insurance['children'].value_counts()

0    574
1    324
2    240
3    157
4     25
5     18
Name: children, dtype: int64

Реализуем mean-target encoding, заменив колонку children колонкой children_category

In [82]:
insurance['children'] = insurance['children'].map(insurance.groupby(['children'])['charges_coef'].mean())
insurance = insurance.rename(columns={'children': 'children_category'})
insurance.head(10)

,age,sex,bmi,children_category,smoker,region,charges_coef
id,,,,,,,
0,19,0,27.900,406.241092,1,1,605.194409
1,18,1,33.770,416.672842,0,0,51.097196
2,28,1,33.000,499.907585,0,0,134.832182
3,33,1,22.705,406.241092,0,2,968.265607
4,32,1,28.880,406.241092,0,2,133.893878
5,31,0,25.740,406.241092,0,0,145.944895
6,46,0,33.440,416.672842,0,0,246.429115
7,37,0,27.740,499.907585,0,2,262.491190
8,37,1,29.830,488.251312,0,3,214.764019


In [83]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X = insurance.drop(['charges_coef'], axis=1)
y = insurance['charges_coef']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model=LinearRegression()
model.fit(X_train, y_train)
for column, coef in zip(X_train.columns, model.coef_):
    print(column, coef)
print(model.intercept_)

age 8.602585028212745
sex -9.426100394120667
bmi -2.7463309109875524
children_category 0.3369227995148387
smoker 745.9978862226509
region 7.950568246541823
-123.38073324431565


In [84]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

features = X_test
y_pred = model.predict(features) # предсказание
y_true = y_test # истина

print(f"MSE = {mean_squared_error(y_true, y_pred)}")
print(f"MAE = {mean_absolute_error(y_pred, y_true)}")
print(f"r2_score  = {r2_score(y_true, y_pred)}")

MSE = 26229.988366289814
MAE = 110.61341691093047
r2_score  = 0.8136289547696308


In [85]:
df = pd.DataFrame(y_true.copy())
df['prediction'] = y_pred
df.head(15)

,charges_coef,prediction
id,,
764,361.273813,382.951416
887,175.622112,256.640496
890,1090.979474,1252.120511
1293,361.308742,376.539188
259,1057.339969,841.750308
1312,105.740303,190.200610
899,94.041255,131.007764
752,374.898719,466.431958
1286,215.883464,230.731177


Исходя из метрик можно сказать, что моя модель не очень хорошо предсказывает коэффициент стоимости страховки, ошибаясь в среднем на 111 пунктов (MAE).

# ЛР №4

In [96]:
import numpy as np
import lightgbm as lgb

insurance_for_lgb['children'] = insurance_for_lgb['children'].astype('category')
children_features = np.unique(insurance_for_lgb['children'].values, axis=0)

X = insurance_for_lgb.drop(['charges_coef'], axis=1)
y = insurance_for_lgb['charges_coef']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

model = lgb.LGBMRegressor(num_leaves=31,  # Количество листьев в дереве
                          learning_rate=0.1,  # Скорость обучения
                          n_estimators=100,  # Количество деревьев
                          categorical_feature=children_features,  # Список категориальных признаков
                          random_state=42)  # Случайное начальное состояние для воспроизводимости
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print()
print(f"MSE: {np.mean((y_test - y_pred) ** 2)}")
print(f"MAE = {mean_absolute_error(y_pred, y_test)}")
print(f"r2_score  = {r2_score(y_true, y_pred)}")
print()


MSE: 23354.775127448884
MAE = 86.67762517599023
r2_score  = 0.8340581097162509



/usr/local/lib/python3.9/dist-packages/lightgbm/basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
/usr/local/lib/python3.9/dist-packages/lightgbm/basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Градиентный бустинг оказался не намного лучше обычной регрессии (-24 пункта), поэтому можно предположить, что по заданным данным трудно довольно точно предсказать коэффициент стоимости.